# Building Data Engineering Pipelines in Python

## Ingesting Data
After seeing this chapter, you will be able to explain what a data platform is, how data ends up in it, and how data engineers structure its foundations. You will be able to ingest data from a RESTful API into the data platform’s data lake using a self-written ingestion pipeline, made using Singer’s taps and targets.

### Components of a data platform
1. #### Dashboards providing business value

> In the video exercise, you saw how a data lake is comprised of several zones. The landing zone contains raw data, the clean zone contains clean data, and the business zone contains domain-specific data, usually related to solving business problems.

> If a dashboard had to be built showing a sales forecast based on the inventory of a warehouse, where would this dashboard get its data from?

##### Answer:
- [ ] From the warehouse management tool
- [ ] From the landing zone
- [ ] From the clean zone
- [x] From the business zone

2. #### Snapshots in a data lake

> In some cases, access to the data lake is provided in the form of a file system, which can be mapped transparently to your working machine. In this exercise, the data lake is available to you under ~/workspace/mnt/data_lake.

> Find out what data file is currently in the business layer, using the commands ls to list the contents of the directory you’re in and cd to change the directory you’re in.

##### Answer:
- [ ] Appreciation scores of a product, as a JSON file.
- [ ] Appreciation scores of a product, as a SQL file.
- [x] A snapshot of customer viewing behavior, in the form of a Parquet file.
- [ ] A snapshot of customer viewing behavior, in the form of a CSV file.

3. #### The data catalog

> To navigate a data lake, a data catalog is typically provided. It abstracts the details of where to find the data and in what format it is stored. Some catalogs hold even more metadata.

> For developers, referencing a data catalog removes hardcoded parts from code. Generally you’re not interested in the details of loading the data, you just want to do useful things with it.

> A very simple data catalog, named catalog, has been prepared for you. It’s a Python dictionary mapping names of datasets to subclasses of DataLinks, a class that you will be unfamiliar with. In the real world, you will often come across classes that you haven’t encountered before. A DataLink is a custom class that simply encapsulates some metadata (like location, file type and presence of headers). Call the .read() method of the catalog’s diaper_reviews to retrieve the actual dataset (so not the metadata stored in the catalog) and inspect this returned object. What type is it?

##### Answer:

- [ ] A list
- [ ] A Pandas DataFrame
- [x] A Spark DataFrame
- [ ] A dictionary
- [ ] A FileSystemDataLink

---

### Introduction to data ingestion with Singer
1. #### Working with JSON

> Because JSON is ubiquitous, you should be able to read and write this format. As you will see in the next lesson, many configuration files in Singer hold JSON.

> In this exercise, you will write some configuration details of a database to a JSON file. Doing this is good practice by the way, as it keeps hardcoded parts out of your code, which allows other people to reference the same configuration files using other languages even.

> Do not hesitate to refer to the slides, available in the tab on the right of IPython Shell, if you’re stuck.

##### Instructions:
* Import the Python module we need to deal with JSON.
* Open a file with the name *database_config.json* for writing (and only for writing).
* Serialize the `database_address` dictionary as JSON and write it to the open file handle. If you're unsure which arguments the function takes, type `?json.dump` to get more information on what `obj` and `fp` expect.

In [4]:
# Import json
import json

database_address = {
    "host": "10.0.0.5",
    "port": 8456
}

# Open the configuration file in writable mode
with open("data/chapter_1/database_config.json", "w") as fh:
    # Serialize the object in this file handle
    json.dump(obj=database_address, fp=fh)

2. #### Specifying the schema of the data

> You’re given a dataset of pricing details of diapers from several stores. After some inspection, you understand that the products have an identical schema, regardless of the store.

> Since your company is already invested in Stitch, the mother company of Singer, you’ll be writing a custom Singer “tap” to export the different products in a standardized way. To do so, you will need to associate a schema with the actual data.

> Example of the products for a particular shop:


In [ ]:
{'items': [{'brand': 'Huggies',
            'model': 'newborn',
            'price': 6.8,
            'currency': 'EUR',            
            'quantity': 40,
            'date': '2019-02-01',
            'countrycode': 'DE'            
            },
           {…}]

##### Instructions:

* Infer from the example above the name and the data type of each component of the store’s items. Complete the JSON schema object with this information.

* Write that schema, using the write_schema() function, to the "products" stream using the Singer API.

In [ ]:
# Complete the JSON schema
schema = {'properties': {
    'brand': {'type': 'string'},
    'model': {'type': 'string'},
    'price': {'type': 'number'},
    'currency': {'type': 'string'},
    'quantity': {'type': 'integer', 'minimum': 1},  
    'date': {'type': 'string', 'format': 'date'},
    'countrycode': {'type': 'string', 'pattern': "^[A-Z]{2}$"}, 
    'store_name': {'type': 'string'}}}

# Write the schema
singer.write_schema(stream_name='products', schema=schema, key_properties=[])

---

### Running an ingestion pipeline with Singer
1. #### Properly propagating state

> You’re running a Singer tap daily at midnight, to synchronize changes between databases. Your tap, called tap-mydelta, extracts only the records that were updated in this database since your last retrieval. To do so, your tap keeps state: it keeps track of the last record it reported on, which can be derived from the table’s last_updated_on field.

> Imagine this table to reflect the current contents of the database in which updates are sometimes being made.

| id | name | last_updated_on|
|----|-----|-----------------|
| 1 | Adrian | 2019-06-14T12:00:04.000+02:00 |
| 2 | Ruanne | 2019-06-16T18:33:21.000+02:00 |
| 3 | Hillary | 2019-06-12T10:05:12.000+02:00 |

> What should the state message be after you run

> `tap-mydelta --state last_state.json --config db_config.json`

> now, with the contents of `last_state.json` being

> `{"max_last_updated_on": "2019-06-16T18:33:21.000+02:00"}`

##### Answer

- [ ] {"type": "STATE", "value": {"max-last-updated-on": "2019-06-14T12:00:04.000+02:00"}}
- [x] {"type": "STATE", "value": {"max-last-updated-on": "2019-06-16T18:33:21.000+02:00"}}
- [ ] {"type": "STATE", "value": {"max-last-updated-on": "2019-06-12T10:05:12.000+02:00"}}
- [ ] {"type": "STATE", "value": {"max-last-updated-on": "null"}}
- [ ] No state message should be emitted.


2. #### Communicating with an API

> Before diving into this third lesson’s concepts, make sure you remember how URLs are constructed and how to interact with web APIs, from the prerequisite course Importing Data in Python, Part 2.

> The marketing team you are collaborating with has been scraping several websites for customer reviews on consumer products. The dataset is only exposed to you through an internal REST API. You would like to add that data in its entirety to the data lake and store it in a convenient way, say csv. While the data is available over the company’s internal network, you still need to supply the API key that the marketing team has created for your exploration use case: `api_key: scientist007`.

> For technical reasons, the endpoint has been made available to you on localhost:5000. You can “browse” to it, using the well-known `requests` module, by calling `requests.get(SOME_URL)`. You can authenticate to the API using your API key. Simply fill in the template URL `<endpoint>/<api_key>/`.

##### Instructions:
* Fill in the correct API key.
* Create the URL of the web API by completing the template URL above. You need to pass the endpoint first and then the API key.
* Use that URL in the call to `requests.get()` so that you may see what more the API can tell you about itself.

In [ ]:
endpoint = "http://localhost:5000"

# Fill in the correct API key
api_key = "scientist007"

# Create the web API’s URL
authenticated_endpoint = "{}/{}".format(endpoint, api_key)

# Get the web API’s reply to the endpoint
api_response = requests.get(authenticated_endpoint).json()
print(api_response)

2. #### Streaming records

> In an earlier exercise, you codified the schema of products being sold by a store that you got from the marketing team’s REST API. If you don’t remember it, check the `schema` in this exercise. Now it’s time to stream some data to go with it.

> A convenience function, `retrieve_products()`, has been made available to you. It basically executes the last step of the previous exercise: it requires one positional argument, the name of a shop, and will return a list of all products related to that shop.

> This exercise is pushing you to use unpacking, using the `**` operator, as shown in the video. If it’s the first time you encounter the unpacking operator, fear not: there’s a first time for everything. We want to show it to you because you will encounter it in your colleagues’ scripts. Do not hesitate to refer to the slides on the tab to the right of the IPython Shell. Unpacking lets you update a record in one step, but you don’t have to use it. You can add the key-value pair outside of the function call as well, by adding a few lines of code. Use any method you like, as long as the record is defined correctly.

##### Instructions:
* Retrieve the products of the shop called `Tesco`.
* Based on the output of the previous step, use the function `write_record()` to write one of these products to the `products` stream, which is where you also wrote the `schema` to. Make sure to add to the product a key-value pair that is mentioned in the schema, but is missing from the product, so that the record you write complies with the schema.
* Now use the more appropriate function `write_records()` to write all items for all shops exposed by the API. As you don’t know a priori how big the dataset will be, you will be using a generator expression in which you enrich the items with the `store_name` one at a time.


In [ ]:
# Use the convenience function to query the API
tesco_items = retrieve_products("Tesco")

singer.write_schema(stream_name="products", schema=schema,
                    key_properties=[])

# Write a single record to the stream, that adheres to the schema
singer.write_record(stream_name="products", 
                    record={**tesco_items[0], "store_name": "Tesco"})

for shop in requests.get(SHOPS_URL).json()["shops"]:
    # Write all of the records that you retrieve from the API
    singer.write_records(
      stream_name="products", # Use the same stream name that you used in the schema
      records=({**item, "store_name": shop}
               for item in retrieve_products(shop))
    )

3. #### Chain taps and targets

    Welcome to your first IDE exercise! This interface is similar to the one you will encounter outside of DataCamp. On the left of the righthand pane, you have a directory of folders and files. You can open the files and see the code in the right part of the pane. For now, you need to use only one file, `data_lake.conf`, which is already open for you, and run a command in the shell (the bottom panel). Feel free to explore the directories, though it’s not required for this exercise.

    ![editor](https://assets.datacamp.com/production/repositories/4724/datasets/b75af48cc1b12f354b14e20bdd6954988592611c/ide_guidance.png)

    Your company’s data lake, which is file system based, is made available to you under /home/repl/workspace/mnt/data_lake. Your goal is to add a file to it, using the Singer tap we’ve been building over the last few exercises, `tap-marketing-api`, and an already existing Singer target, `target-csv`.

    Your Singer tap has been packaged as `tap-marketing-api` (you can call it like that from the bash shell, which is in the bottom panel). Its output is simply the schema and records you made earlier. You can therefore use it in a shell pipeline, like so: `tap-marketing-api | some-target`. Remember, some taps and targets can be configured through configuration and state files in which case you should use those flags and pass in relevant files, e.g. `some-tap | target-csv --config some_config_file`.


##### Instructions:

Create a Singer pipeline that will create a file in the landing area of the data lake, holding the output of `tap-marketing-api` in CSV format. To do so:
    
* In the shell, pipe the output of your Singer "tap", `tap-marketing-api`, to `target-cs`
* Pass the configuration file *data_lake.conf* (located in the ingest folder of the data lake) to `target-csv`, using the `--config` flag. The configuration file specifies where the CSV file should be written to and should not be changed.

    Execute your command under `./data/chapter_1/` directory.
    
##### Answer
`tap-marketing-api | target-csv --config ingest/data_lake.conf`